In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

%cd /content/drive/MyDrive/Projects/Dastpokht/comment_analyzer/phi-3/comment_analyzer

Mounted at /content/drive/
/content/drive/MyDrive/Projects/Dastpokht/comment_analyzer/phi-3/comment_analyzer


In [ ]:
!ls

analyzer  data	DB_utils  main.py  models  __pycache__	requirements.txt  results


In [ ]:
%%capture
!pip install -q -r requirements.txt
!pip install accelerate

In [ ]:
import os
os.environ['POSTGRES_USER'] = 'postgres'
os.environ['POSTGRES_PASSWORD'] = '...'
os.environ['POSTGRES_DB'] = 'postgres'
os.environ['POSTGRES_SERVER'] = '22ba3b43-cffb-471a-8161-5ed71094baeb.hsvc.ir'
os.environ['POSTGRES_PORT'] = '30667'

# Import libraries

In [ ]:
import pandas as pd

# Helper fucntions

In [ ]:
def get_comments(df):
    comments = []
    for i in range(len(df)):
        comment_dict = {
            'comment_id': df.comment_id[i],
            'foods': df.foods[i],
            'comment_text': df.comment_text[i]
        }
        comments.append(comment_dict)
    return comments

def get_comment_from_analyze(vendor_id, comment_id):
    df_1 = pd.read_csv(f"data/all_new_comments_vendor_id={vendor_id}.csv", encoding="utf-8")
    results = []
    for i in range(len(df_1)):
        if int(df_1['comment_id'][i]) == int(comment_id):
            results.append(df_1['comment_id'][i])
    if results:
        return True
    else:
        False


In [ ]:
import os

def extract_vendor_id(filename):
  """Extracts the vendor ID from a filename.

  Args:
      filename: The filename to extract the vendor ID from.

  Returns:
      The vendor ID as a string, or None if no vendor ID is found.
  """
  parts = filename.split("vendor_id=")
  if len(parts) == 2:
    return parts[1].split(".")[0]  # Get part after '=' and before '.'
  else:
    return None

# Get filenames from the folder
folder_path = "data/"  # Replace with your actual folder path
filenames = os.listdir(folder_path)

# Extract vendor IDs
vendor_ids = []
for filename in filenames:
  vendor_id = extract_vendor_id(filename)
  if vendor_id:
    vendor_ids.append(vendor_id)

# Print the extracted vendor IDs
print(vendor_ids)


['267355', '267355', '85751', '85751', '42111', '42111', '360644', '360644', '31343', '31343', '364527', '364527', '28742', '28742', '342906', '342906', '262667', '262667', '322076', '322076', '305699', '305699', '247134', '247134', '271855', '271855', '34392', '34392', '234470', '234470', '64807', '64807', '26926', '26926', '358426', '358426', '105892', '105892', '32817', '32817', '88539', '88539', '57607', '57607', '351592', '351592', '55804', '55804', '322549', '322549', '60396', '60396', '306720', '306720', '299324', '299324', '79514', '79514', '48482', '48482', '44241', '44241', '17795', '17795', '16023', '16023', '350471', '350471', '55979', '55979', '50310', '50310', '289771', '289771', '16023']


In [ ]:
# from DB_utils.ERM.storage.interfaces.get_data import get_product, get_comments, get_vendor, get_vendors, \
#     get_comment_from_analyze
from DB_utils.ERM.storage.interfaces.add_data import add_analyze
import requests
import json
from analyzer import analyze  # Import the analyze function

def fetch_and_analyze_comments(vendor_id):
    df = pd.read_csv(f"data/comment_vendor_id={vendor_id}.csv", encoding="utf-8")
    comments = get_comments(df)
    counter = 1
    df_1 = pd.read_csv(f"data/all_new_comments_vendor_id={vendor_id}.csv", encoding="utf-8")
    df = df.reindex(columns= df.columns.tolist() + ['response'] +['raw_results'] + ['details'])
    if len(comments) > 10:
        for index, comment in enumerate(comments[:50]):
            if get_comment_from_analyze(vendor_id, comment['comment_id']):
                    pass
            data = {"comment_text": comment['comment_text']}
            data['comment_id'] = int(comment['comment_id'])
            materials = df_1["foods"][df_1['comment_id'] == data['comment_id']].values.tolist()
            if len(materials) > 0 :
                data['materials'] = [{eval(m)['product_title'] : eval(m)['product_description']} for m in materials]
                data["foods"] = materials
                detail = materials
                try:
                    analysis_results, raw_results = analyze(data)
                    print(f'request number {counter}, AI response is : {analysis_results}')
                    counter += 1
                    # try:
                    #   raw_results = raw_results.split("```json")[1]
                    #   raw_results = raw_results.split("```")[0]
                    # except:
                    #   pass
                    df.loc[df['comment_id'] == data['comment_id'], ['raw_results']] = str(raw_results)
                    df.loc[df['comment_id'] == data['comment_id'], ['details']] = str(materials)
                    df.loc[df['comment_id'] == data['comment_id'], ['response']] = str(analysis_results)
                    print("correct output")
                except:
                    print("Bad output format")
                    pass
    return df.to_csv(f"results/comment_vendor_id={vendor_id}_50_first_rows.csv", encoding="utf-8")




In [ ]:
x = fetch_and_analyze_comments(16023)

Loading model from local storage...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

...

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading tokenizer from local storage...
{
    "foods overall rating": 5,

    "qualities": [
        {
            "food_title": "چلو خورشت قیمه بادمجان",
            "food_rate": 5
        }
    ],

    "services": []
}
{
    "foods overall rating": 5,
    "qualities": [
        {
            "food_title": "چلو خورشت قورمه سبزی",
            "food_rate": 5
        }
    ],
    "services": []
}
request number 35, AI response is : {0: {'qualities': [{'food_title': 'چلو خورشت قیمه بادمجان', 'food_rate': 5}], 'foods overall rating': 5, 'services': []}, 1: {'qualities': [{'food_title': 'چلو خورشت قورمه سبزی', 'food_rate': 5}], 'foods overall rating': 5, 'services': []}}
correct output
Loading model from local storage...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading tokenizer from local storage...
{
    "foods overall rating": 4,
    "qualities": [
        {
            "food_title": "چلو خورشت قیمه بادمجان",
            "food_rate": 4
        },
        {
            "food_title": "بادمجان",
            "food_rate": 2
        }
    ],
    "services": []
}
request number 36, AI response is : {0: {'qualities': [{'food_title': 'چلو خورشت قیمه بادمجان', 'food_rate': 4}], 'foods overall rating': 4, 'services': []}}
correct output


In [ ]:
counter = 1
# drop duplicates
vendor_ids = list(set(vendor_ids))
print(len(vendor_ids))
for vendor in vendor_ids[:5]:
    print(f"Length vendor ids: {len(vendor_ids)}")
    print(f"[{counter}] process started for vendor id: {vendor}")
    fetch_and_analyze_comments(vendor)
    counter += 1
    print('#'*50)



# New models


## Aya_huggingface

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="CohereForAI/aya-23-8B")


config.json:   0%|          | 0.00/640 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/17.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.5M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
messages = [
    {"role": "user", "content": """Suppose that we are working in the domain of a food delivery app.

the input data ( UserOrder, FoodIngredients, UserComment,) is only within the three backticks

UserComment expresses the user's satisfaction with the food quality and services delivered.

Having the context provided,
your task is to evaluate the food quality and services delivered based on UserComment and assign a score per each food and services delivered (an integer between 0 (for most disatisfies) and 5 (for most satisfied))considering the following rules:

Handle Mixed Sentiments:
If UserComment contains mixed sentiments (like positive followed by negative), consider the overall tone of the comment at its conclusion.
Focus on how the UserComment ends to determine if it’s generally positive or negative.
This prevents confusion over mixed messages within the comment.

1. Rate Food quality:

Analyze a user's comment to rate each food item in the UserOrder and the overall food experience.

Task:

1. Determine if the comment expresses satisfaction or dissatisfaction with the quality of any food item in the UserOrder.
2. Check if the comment directly mentions specific foods or food ingredients.
3. If the comment mentions a specific food from the UserOrder, rate that food based on the sentiment expressed.
4. If the comment mentions food ingredients, identify the food(s) containing those ingredients using the FoodIngredients list and rate that food based on the sentiment.
5. If the comment provides an overall assessment of the food experience without specifying a particular food item, only rate the "foods overall rating" score as the general order rating.
6. If the comment does not mention a specific food item, set the score for that food item to 'Nan'.
7. Apply the 'Handle Mixed Sentiments' rule when rating each food item and the overall food experience.

note : "foods overall rating" is not the average value of the food items


Important: if the comment says that a food item was raw (in persian:  خام  ) consider a low score for that food item. also  افتضاح  means awful.


Note: Ensure that each food item in the UserOrder receives a score, and an additional rating is provided for the "foods overall rating".


2. rate Service Quality :

Analyze a user's comment to rate  service categories:


1. Staff Behavior : Look for mentions of driver, delivery person,

2. Delivery Time: Analyze the comment for any mention of time related words or phrases  as "late", "fast", "on time", "delayed", "soon"  If found, assign a score.

3. Food Temperature: Look for words indicating temperature such as "hot", "cold", "warm", Assign a score based on the sentiment expressed about the food temperature

4. Delivered Food not as Ordered : Look for phrases indicating the food was not as ordered such as "wrong order", "not what I ordered", etc. Assign a score based on the sentiment expressed

5. packaging : Identify comments that mention the condition of the packaging when it arrived. Keywords might include “well-packed”, “leaking”, or “damaged”.

6. Cost of Delivery: Look for any mention of the delivery fee or cost. Phrases like “expensive delivery”, “cheap delivery”, or “free delivery” can be relevant.

7. Cost per Value of Foods: This involves comments about whether the user felt they got their money’s worth for the food they ordered. Relevant phrases could be ( "expensive"= " گران " ) “worth the price”, “overpriced”, or “good value”.


Important: food quality is not  part of the service ratings.

Remember, if the comment doesn’t mention services or fit any category, or if the sentiment is unclear, keep the service part empty in the JSON.
do not add any new categories to the service ratings.

3. Evaluate Food Temperature:

Analyze the UserComment to evaluate the temperature of the food.

Task:

1. Check if the UserComment mentions the temperature of a specific food item.
2. Identify the food item being discussed and determine its typical serving temperature (hot or cold).
3. Evaluate the UserComment to determine if the food temperature was satisfactory.
        If the food is typically served hot, but the comment indicates it was ' سرد ' (cold) assign a low temperature score.
        If the food is typically served warm, and the comment indicates it was ' گرم ' (warm), assign a higher temperature score.
        If the food temperature matches its typical serving temperature, assign a higher temperature score.
4. Update the services ratings with the evaluated food temperature score.

Note: Consider the typical serving temperature of different food types, such as hot for fast foods and main dishes, and cold for drinks and salads.


ATTENTION :
As you see, the UserComment is in Persian. Thus, be mindful of any different Persian idioms or slang used and their actual meanings.
don't translate food names


Based on the analysis, generate JSON for foods and services in this format:

{{
    "foods overall rating": <foods overall quality rating>,

    qualities: [
        {{
            "food_title": <food_title>,
            "food_rate": <food_rate>
        }}
    ],


    services: [
        {{
            {{
            "part_of_comment": <part_of_comment>,
            "service_rate": <service_rate>,
            "service_category": <service_category>
        }}
        }}

    ]
}}

```
Here is the list of foods and beverages ordered by a user, let's call it 'UserOrder':

UserOrder: {[{'title': 'خوراک کباب کوبیده (2 سیخ)'}]}

If available, here is the  of ingredients for each or some food included in the UserOrder (called 'FoodIngredients'):
FoodIngredientsFoodIngredients: {'2 سیخ کباب کوبیده 120 گرمی، دورچین: خیارشور و کلم قرمز'}

here is the user's comment (review) about their experience with UserOrder (called 'UserComment'):

UserComment: {خوب وخوشمزه بودوزود رسید}"""},
]


In [ ]:
import os
os.environ['HF_TOKEN'] = "..."
os.environ['HUGGINGFACEHUB_API_TOKEN'] = "..."

In [ ]:
from getpass import getpass
HUGGINGFACEHUB_API_TOKEN = getpass()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN

··········


In [ ]:
import transformers
import torch

# model_id = "myrkur/paya"
model_id = "CohereForAI/aya-23-8B"
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

# messages = [
#     {"role": "user", "content": "علم بهتر است یا ثروت؟"},
# ]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
]

outputs = pipeline(
    prompt,
    max_new_tokens=1500,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.4,
    top_p=0.9,
    repetition_penalty=1.1
)
print(outputs[0]["generated_text"][len(prompt):])


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Based on the provided information and instructions, here is the JSON representation of the food quality and service ratings:

```json
{
  "foods overall rating": 4,
  "qualities": [
    {
      "food_title": "کباب کوبیده",
      "food_rate": 5
    },
    {
      "food_title": "خوراک کباب",
      "food_rate": 4
    }
  ],
  "services": [
    {
      "part_of_comment": "سودت رسید",
      "service_rate": 5,
      "service_category": "Delivery Time"
    },
    {
      "part_of_comment": "خدمات خوب بود",
      "service_rate": 4,
      "service_category": "Staff Behavior"
    },
    {
      "part_of_comment": null,
      "service_rate": 3,
      "service_category": "Food Temperature"
    },
    {
      "part_of_comment": null,
      "service_rate": 4,
      "service_category": "Delivered Food Not As Ordered"
    },
    {
      "part_of_comment": null,
      "service_rate": 4,
      "service_category": "Packaging"
    },
    {
      "part_of_comment": null,
      "service_rate": 4,
      "ser

In [ ]:
messages = [
    {"role": "user", "content": """Suppose that we are working in the domain of a food delivery app.

the input data ( UserOrder, FoodIngredients, UserComment,) is only within the three backticks

UserComment expresses the user's satisfaction with the food quality and services delivered.

Having the context provided,
your task is to evaluate the food quality and services delivered based on UserComment and assign a score per each food and services delivered (an integer between 0 (for most disatisfies) and 5 (for most satisfied))considering the following rules:

Handle Mixed Sentiments:
If UserComment contains mixed sentiments (like positive followed by negative), consider the overall tone of the comment at its conclusion.
Focus on how the UserComment ends to determine if it’s generally positive or negative.
This prevents confusion over mixed messages within the comment.

1. Rate Food quality:

Analyze a user's comment to rate each food item in the UserOrder and the overall food experience.

Task:

1. Determine if the comment expresses satisfaction or dissatisfaction with the quality of any food item in the UserOrder.
2. Check if the comment directly mentions specific foods or food ingredients.
3. If the comment mentions a specific food from the UserOrder, rate that food based on the sentiment expressed.
4. If the comment mentions food ingredients, identify the food(s) containing those ingredients using the FoodIngredients list and rate that food based on the sentiment.
5. If the comment provides an overall assessment of the food experience without specifying a particular food item, only rate the "foods overall rating" score as the general order rating.
6. If the comment does not mention a specific food item, set the score for that food item to 'Nan'.
7. Apply the 'Handle Mixed Sentiments' rule when rating each food item and the overall food experience.

note : "foods overall rating" is not the average value of the food items


Important: if the comment says that a food item was raw (in persian:  خام  ) consider a low score for that food item. also  افتضاح  means awful.


Note: Ensure that each food item in the UserOrder receives a score, and an additional rating is provided for the "foods overall rating".


2. rate Service Quality :

Analyze a user's comment to rate  service categories:


1. Staff Behavior : Look for mentions of driver, delivery person,

2. Delivery Time: Analyze the comment for any mention of time related words or phrases  as "late", "fast", "on time", "delayed", "soon"  If found, assign a score.

3. Food Temperature: Look for words indicating temperature such as "hot", "cold", "warm", Assign a score based on the sentiment expressed about the food temperature

4. Delivered Food not as Ordered : Look for phrases indicating the food was not as ordered such as "wrong order", "not what I ordered", etc. Assign a score based on the sentiment expressed

5. packaging : Identify comments that mention the condition of the packaging when it arrived. Keywords might include “well-packed”, “leaking”, or “damaged”.

6. Cost of Delivery: Look for any mention of the delivery fee or cost. Phrases like “expensive delivery”, “cheap delivery”, or “free delivery” can be relevant.

7. Cost per Value of Foods: This involves comments about whether the user felt they got their money’s worth for the food they ordered. Relevant phrases could be ( "expensive"= " گران " ) “worth the price”, “overpriced”, or “good value”.


Important: food quality is not  part of the service ratings.

Remember, if the comment doesn’t mention services or fit any category, or if the sentiment is unclear, keep the service part empty in the JSON.
do not add any new categories to the service ratings.

3. Evaluate Food Temperature:

Analyze the UserComment to evaluate the temperature of the food.

Task:

1. Check if the UserComment mentions the temperature of a specific food item.
2. Identify the food item being discussed and determine its typical serving temperature (hot or cold).
3. Evaluate the UserComment to determine if the food temperature was satisfactory.
        If the food is typically served hot, but the comment indicates it was ' سرد ' (cold) assign a low temperature score.
        If the food is typically served warm, and the comment indicates it was ' گرم ' (warm), assign a higher temperature score.
        If the food temperature matches its typical serving temperature, assign a higher temperature score.
4. Update the services ratings with the evaluated food temperature score.

Note: Consider the typical serving temperature of different food types, such as hot for fast foods and main dishes, and cold for drinks and salads.


ATTENTION :
As you see, the UserComment is in Persian. Thus, be mindful of any different Persian idioms or slang used and their actual meanings.
don't translate food names


Based on the analysis, generate JSON for foods and services in this format:

{{
    "foods overall rating": <foods overall quality rating>,

    qualities: [
        {{
            "food_title": <food_title>,
            "food_rate": <food_rate>
        }}
    ],


    services: [
        {{
            {{
            "part_of_comment": <part_of_comment>,
            "service_rate": <service_rate>,
            "service_category": <service_category>
        }}
        }}

    ]
}}

```
Here is the list of foods and beverages ordered by a user, let's call it 'UserOrder':

UserOrder: [{'title': 'چلو خورشت قیمه بادمجان'}]

If available, here is the  of ingredients for each or some food included in the UserOrder (called 'FoodIngredients'):
FoodIngredientsFoodIngredients: {به همراه 70 گرم گوشت گوساله، بادمجان سرخ کرده، چلو کره ایرانی'}

here is the user's comment (review) about their experience with UserOrder (called 'UserComment'):

UserComment: {من قیمه بادمجان سفارش دادم برام قیمه سیبزمینی فرستادن و سیب زمینی هم نبود چیپس خلالی بود و روی برنج یه تیکه خیلی کوچیک بادمجان گذاشتن}"""},
]


In [ ]:
messages = [
    {"role": "user", "content": """Suppose that we are working in the domain of a food delivery app.

the input data ( UserOrder, FoodIngredients, UserComment,) is only within the three backticks

UserComment expresses the user's satisfaction with the food quality and services delivered.

Having the context provided,
your task is to evaluate the food quality and services delivered based on UserComment and assign a score per each food and services delivered (an integer between 0 (for most disatisfies) and 5 (for most satisfied))considering the following rules:

Handle Mixed Sentiments:
If UserComment contains mixed sentiments (like positive followed by negative), consider the overall tone of the comment at its conclusion.
Focus on how the UserComment ends to determine if it’s generally positive or negative.
This prevents confusion over mixed messages within the comment.

1. Rate Food quality:

Analyze a user's comment to rate each food item in the UserOrder and the overall food experience.

Task:

1. Determine if the comment expresses satisfaction or dissatisfaction with the quality of any food item in the UserOrder.
2. Check if the comment directly mentions specific foods or food ingredients.
3. If the comment mentions a specific food from the UserOrder, rate that food based on the sentiment expressed.
4. If the comment mentions food ingredients, identify the food(s) containing those ingredients using the FoodIngredients list and rate that food based on the sentiment.
5. If the comment provides an overall assessment of the food experience without specifying a particular food item, only rate the "foods overall rating" score as the general order rating.
6. If the comment does not mention a specific food item, set the score for that food item to 'Nan'.
7. Apply the 'Handle Mixed Sentiments' rule when rating each food item and the overall food experience.

note : "foods overall rating" is not the average value of the food items


Important: if the comment says that a food item was raw (in persian:  خام  ) consider a low score for that food item. also  افتضاح  means awful.


Note: Ensure that each food item in the UserOrder receives a score, and an additional rating is provided for the "foods overall rating".


2. rate Service Quality :

Analyze a user's comment to rate  service categories:


1. Staff Behavior : Look for mentions of driver, delivery person,

2. Delivery Time: Analyze the comment for any mention of time related words or phrases  as "late", "fast", "on time", "delayed", "soon"  If found, assign a score.

3. Food Temperature: Look for words indicating temperature such as "hot", "cold", "warm", Assign a score based on the sentiment expressed about the food temperature

4. Delivered Food not as Ordered : Look for phrases indicating the food was not as ordered such as "wrong order", "not what I ordered", etc. Assign a score based on the sentiment expressed

5. packaging : Identify comments that mention the condition of the packaging when it arrived. Keywords might include “well-packed”, “leaking”, or “damaged”.

6. Cost of Delivery: Look for any mention of the delivery fee or cost. Phrases like “expensive delivery”, “cheap delivery”, or “free delivery” can be relevant.

7. Cost per Value of Foods: This involves comments about whether the user felt they got their money’s worth for the food they ordered. Relevant phrases could be ( "expensive"= " گران " ) “worth the price”, “overpriced”, or “good value”.


Important: food quality is not  part of the service ratings.

Remember, if the comment doesn’t mention services or fit any category, or if the sentiment is unclear, keep the service part empty in the JSON.
do not add any new categories to the service ratings.

3. Evaluate Food Temperature:

Analyze the UserComment to evaluate the temperature of the food.

Task:

1. Check if the UserComment mentions the temperature of a specific food item.
2. Identify the food item being discussed and determine its typical serving temperature (hot or cold).
3. Evaluate the UserComment to determine if the food temperature was satisfactory.
        If the food is typically served hot, but the comment indicates it was ' سرد ' (cold) assign a low temperature score.
        If the food is typically served warm, and the comment indicates it was ' گرم ' (warm), assign a higher temperature score.
        If the food temperature matches its typical serving temperature, assign a higher temperature score.
4. Update the services ratings with the evaluated food temperature score.

Note: Consider the typical serving temperature of different food types, such as hot for fast foods and main dishes, and cold for drinks and salads.


ATTENTION :
As you see, the UserComment is in Persian. Thus, be mindful of any different Persian idioms or slang used and their actual meanings.
don't translate food names


Based on the analysis, generate JSON for foods and services in this format:

{{
    "foods overall rating": <foods overall quality rating>,

    qualities: [
        {{
            "food_title": <food_title>,
            "food_rate": <food_rate>
        }}
    ],


    services: [
        {{
            {{
            "part_of_comment": <part_of_comment>,
            "service_rate": <service_rate>,
            "service_category": <service_category>
        }}
        }}

    ]
}}

```
Here is the list of foods and beverages ordered by a user, let's call it 'UserOrder':

UserOrder: [{'title': 'چلو ماهی قزل آلا'}]

If available, here is the  of ingredients for each or some food included in the UserOrder (called 'FoodIngredients'):
FoodIngredientsFoodIngredients: {'چلو ماهی قزل آلا', 'product_description': '400 گرم ماهی قزل سرخ شده، چلو کره ایرانی'}

here is the user's comment (review) about their experience with UserOrder (called 'UserComment'):

UserComment: {با اینکه در نظرات قبلی هم گفته شده بود ولی موارد متاسفانه تکرار شده از جمله: روغن زیاد ماهی، کیفیت پایین برنج، قرار دادن ترشی زیر ماهی و بد مزه و روغنی شدن آن، بسته بندی نامناسب}"""},
]
pipe(messages)